In [1]:
import pandas as pd
from plotly.subplots import make_subplots
import numpy as np
import helper as h 
from IPython.display import display, Markdown
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
TZ = 'Europe/Berlin'
# TZ = 'America/Los_Angeles'

TARGET_DELTA_FASTING = 4
TARGET_DELTA_FIRST_MEAL = 1
TARGET_DELTA_LAST_MEAL = 3
TARGET_DELTA_SLEEP = 7
DT_DEEP_FAST_IN_HOURS = 12 # this is the time after which the deep fasting state starts
MIN_GAP_BETWEEN_SESSIONS_IN_MINUTES = DT_DEEP_FAST_IN_HOURS*60

In [3]:
# load sleep data
df_sleep = h.load_sleep_data_from_supabase()
df_sleep_sessions = h.identify_sessions(df_sleep, min_gap_between_sessions_in_minutes=30, min_duration_of_session_in_minutes=60, add_sleep_duration_in_hours=True)

# load food data
df_eat = h.load_eat_data_from_supabase()
df_eat_sessions = h.identify_sessions(df_eat, min_gap_between_sessions_in_minutes=MIN_GAP_BETWEEN_SESSIONS_IN_MINUTES, min_duration_of_session_in_minutes=0)

# evaluate deep fasting sessions
df_deep_fast_sessions = h.evaluate_deep_fast_sessions(df_eat_sessions, DT_DEEP_FAST_IN_HOURS)

# evaluate delta to first and last meal
df_first_and_last_meal = h.evaluate_delta_to_first_and_last_meal(df_sleep_sessions, df_eat)

# evaluate data for visualization
df_sleep_sessions_viz = h.process_for_visualization(df_sleep_sessions, TZ)
df_eat_viz = h.process_for_visualization(df_eat, TZ)
df_eat_sessions_viz = h.process_for_visualization(df_eat_sessions, TZ)
df_deep_fast_sessions_viz = h.process_for_visualization(df_deep_fast_sessions, TZ)

df_deep_fast_viz = h.process_deep_fast_sessions_for_viz(df_deep_fast_sessions, timezone=TZ)
df_first_meal_viz, df_last_meal_viz = h.process_first_and_last_meal_data_for_viz(df_first_and_last_meal, timezone=TZ)
df_sleep_duration_viz = h.process_sleep_sessions_for_viz(df_sleep_sessions, TZ)

# calculate score
df_score = h.calculate_score(
    df_deep_fast_viz, 
    df_first_meal_viz, 
    df_last_meal_viz, 
    df_sleep_duration_viz,
    target_delta_fasting = TARGET_DELTA_FASTING,
    target_delta_first_meal = TARGET_DELTA_FIRST_MEAL,
    target_delta_last_meal = TARGET_DELTA_LAST_MEAL,
    target_delta_sleep = TARGET_DELTA_SLEEP,
)

In [4]:
def get_current_and_last_score(df_score):

    # Current score (prediction)
    last_valid_values = []
    cols =["score_fasting", "score_first_meal", "score_last_meal", "score_sleep"]
    for c in cols:
        last_valid_values.append(df_score[c].dropna().iloc[-1])
    score_current = np.mean(last_valid_values)
    ts_current = df_score.date.dropna().iloc[-1]

    # Last valid
    row = df_score.dropna().iloc[-1]
    score_last = row.score
    ts_last = row.date
    
    return score_current, ts_current, score_last, ts_last

score_current, ts_current, score_last, ts_last = get_current_and_last_score(df_score)


In [5]:
fig = make_subplots(rows=6, cols=1, shared_xaxes=True, vertical_spacing=0.025)

# Sleep sessions
fig.add_bar(x=pd.to_datetime(df_sleep_sessions_viz.date),
    y=df_sleep_sessions_viz.dh,
    base=df_sleep_sessions_viz.h1,
    opacity=0.8,
    marker_color='gray',
    hovertemplate=df_sleep_sessions_viz.info_html,
    name="Asleep",
    row=1, col=1
)

# East Sessions
fig.add_bar(x=pd.to_datetime(df_eat_sessions_viz.date),
    y=df_eat_sessions_viz.dh,
    base=df_eat_sessions_viz.h1,
    opacity=0.2,
    marker_color='darkgreen',
    hovertemplate=df_eat_sessions_viz.info_html,
    name="Feeding Windows",
    row=1, col=1
)

fig.add_bar(x=pd.to_datetime(df_eat_viz.date),
    y=df_eat_viz.dh,
    base=df_eat_viz.h1,
    opacity=1.0,
    marker_color='darkgreen',
    hovertemplate=df_eat_viz.info_html,
    name="Meals",
    row=1, col=1
)

# Fast windows
fig.add_bar(x=pd.to_datetime(df_deep_fast_sessions_viz.date),
    y=df_deep_fast_sessions_viz.dh,
    base=df_deep_fast_sessions_viz.h1,
    opacity=0.5,
    marker_color='aqua',
    hovertemplate=df_deep_fast_sessions_viz.info_html,
    name="Deep Fast",
    row=1, col=1
)

fig.add_hline(
    y=7,
    line_dash="dot",
    line_color="white",
    row=5, col=1
)

for df, name, color, row in zip(
    [df_deep_fast_viz, df_first_meal_viz, df_last_meal_viz, df_sleep_duration_viz], 
    ["Deep Fasting Duration", "First Meal After Sleep", "Last Meal Before Sleep", "Sleep Duration"],
    ["aqua", "orange", "yellow", "white"],
    [2, 3, 4, 5]
):
        
    fig.add_scatter(
        x=df.date,
        y=df.delta_in_hours,
        mode='lines+markers',
        name=name,
        opacity=0.2,
        marker_color=color,
        row=row, col=1
    )

    fig.add_scatter(
        x=df.date,
        y=df.mean_delta_in_hours,
        mode='lines',
        name=name + (" Weekly Average"),
        marker_color=color,
        row=row, col=1
    )
    
    
    

for y, row in zip([TARGET_DELTA_FASTING, TARGET_DELTA_FIRST_MEAL, TARGET_DELTA_LAST_MEAL, TARGET_DELTA_SLEEP], [2, 3, 4, 5]):
    fig.add_hline(
        y=y,
        line_dash="dot",
        line_color="white",
        line_width=1,
        row=row, col=1
    )

fig.add_scatter(
    x=df_score.date,
    y=df_score.score,
    mode='lines',
    name="Score",
    opacity=1,
    marker_color="red",
    row=6, col=1
)

fig.add_scatter(
    x=[ts_last,ts_current],
    y=[score_last, score_current],
    mode='lines',
    line=dict(color='red', dash='dot'),
    row=6, col=1,
    showlegend=False
)

fig.add_scatter(
    x=[ts_current],
    y=[score_current],
    mode='markers',
    name="Current Score",
    marker=dict(color='red', size=6),
    row=6, col=1
)


# Layout
fig.update_layout(
    barmode='overlay',
    template='plotly_dark',
    bargap=0.05,
    width=1000,
    height=1000,
)

fig.update_xaxes(showline=True, mirror=True, showgrid=True, range=[pd.Timestamp('2024-10-15'), pd.Timestamp.now()])

fig.update_yaxes(showline=True, mirror=True, showgrid=False, range=[0, 24], row=1, col=1)
for row in [2,3,4]:
    fig.update_yaxes(showline=True, mirror=True, showgrid=False, range=[0, 8], row=row, col=1)
fig.update_yaxes(showline=True, mirror=True, showgrid=False, range=[4, 10], row=5, col=1)
fig.update_yaxes(showline=True, mirror=True, showgrid=True, range=[0, 1], tickformat=',.0%', row=6, col=1)


# Example for adding annotations to each subplot using the layout annotations list
fig.update_layout(
    annotations=[
        dict(
            text="Deep Fast",
            x=0, y=1,
            xref="x2 domain",
            yref="y2 domain",
            showarrow=False,
            font=dict(color="aqua", size=12)
        ),
        dict(
            text="First Meal After Sleep",
            x=0, y=1,
            xref="x3 domain", 
            yref="y3 domain",
            showarrow=False,
            font=dict(color="orange", size=12)
        ),
        dict(
            text="Last Meal Before Sleep",
            x=0, y=1,
            xref="x4 domain",
            yref="y4 domain",
            showarrow=False,
            font=dict(color="yellow", size=12)
        ),
        dict(
            text="Sleep Duration",
            x=0, y=1,
            xref="x5 domain",
            yref="y5 domain",
            showarrow=False,
            font=dict(color="white", size=12)
        ),
    ]
)

fig.show()

In [6]:
#longest eating session
print("\nlongest eating session")
idx = df_eat_sessions.duration_in_hours.idxmax()
print(df_eat_sessions.iloc[idx])

#longest deep fasting session
print("\nlongest fasting session")
idx = df_deep_fast_sessions.delta_in_hours.idxmax()
print(df_deep_fast_sessions.iloc[idx])
dt = float(df_deep_fast_sessions.iloc[idx].delta_in_hours + DT_DEEP_FAST_IN_HOURS)
print("overall fasting duration:", dt)


longest eating session
session                                                             81
ts_start                                     2025-01-08 14:43:56+00:00
ts_end                                       2025-01-11 02:21:21+00:00
duration_in_hours                                            59.623611
info_dict            {'session': 81, 'duration_in_hours': 59.623611...
Name: 81, dtype: object

longest fasting session
ts_start                      2025-03-23 07:43:27+01:00
ts_end                        2025-03-23 15:35:17+01:00
delta_in_hours                                 7.863889
info_dict         {'delta_in_hours': 7.863888888888889}
date                                         2025-03-23
Name: 146, dtype: object
overall fasting duration: 19.863888888888887


In [18]:
ts_now = pd.Timestamp.now(tz=TZ)

# (df_deep_fast_sessions.ts_end - ts_now).dt.total_seconds()

mask_current_deep_fast_session = (ts_now - df_deep_fast_sessions.ts_end) < pd.Timedelta(minutes=3)
e = df_deep_fast_sessions[mask_current_deep_fast_session]
if len(e) > 0:
    e = e.iloc[0]
    print(e)
else:
    print("no current deep fast session")


ts_start                       2025-04-06 07:42:57+02:00
ts_end                  2025-04-06 11:14:02.641781+02:00
delta_in_hours                                  3.518234
info_dict         {'delta_in_hours': 3.5182338280555556}
date                                          2025-04-06
Name: 157, dtype: object


In [22]:
e.ts_start, e.delta_in_hours


(Timestamp('2025-04-06 07:42:57+0200', tz='Europe/Berlin'),
 np.float64(3.5182338280555556))